# Práctica 2: Manejo básico de ROS2

## Objetivo

Que el alumno haga uso de las funciones básicas de ROS2 (publicador y suscriptor) y de sus aplicaciones con turtlesim y rviz2

### Metas

Que el alumno:
- Integre texto en Markdown dentro de Jupyter
- Integre código de Python dentro de Jupyter
- Haga uso de la terminal de Ubuntu y contruya un script de bash
- Genere un repositorio en GitHub y comparta su archivo de práctica

### Contribución al perfil del egresado

La siguiente práctica contribuye en los siguientes puntos al perfil del egresado:

#### Aptitudes y habilidades

- Para diseñar, construir, operar y mantener los sistemas mecatrónicos y sus componentes.
- Para crear, innovar o evaluar las tecnologías relacionadas con la mecatrónica.

#### Actitudes profesionales

- Ser creativo e innovador.
- Tener confianza en su preparación académica.
- Comprometido con su actualización, superación y competencia profesional.

#### Actitudes de tipo social

- Promover el cambio en la mentalidad frente a la competitividad internacional.

## Rúbrica de evaluación

### La evaluación de la práctica contará de los siguientes puntos

| Elemento | Porcentaje |
| ------:| ----------- |
| **Cuestionario previo** | 15% | 
| **Desarrollo** | 35% |
| **Análisis de resultados**  | 35% |
| **Conclusiones** | 15% |

### Se evaluará con los siguientes criterios:

| Elemento | Malo | Regular | Bueno |
| ------:| ------ | --------| ------|
| **Cuestionario previo** | El trabajo no contiene cuestionario previo o todas las preguntas son incorrectas (0%)| Al menos la mitad de las preguntas son correctas (8%) |  Todas las preguntas son correctas (15%) |
| **Desarrollo** | El trabajo no contiene desarrollo o su planteamiento no concuerda con lo deseado (0%) | El desarrollo está mal planteado o no llega a los resultados esperados (10%) | El desarrollo tiene un planteamiento adecuado y llega a los resultados esperados (35%) |
| **Análisis de resultados**  | El trabajo no contiene análisis de resultados o la información no se está interpretando correctamente (0%) | La interpretación de los resultados es parcial o desorganizada (10%) | Realiza un correcto análisis de los resultados de forma organizada   (35%) |
| **Conclusiones** | El trabajo no contiene conclusiones o no hacen referencia al trabajo desarrollado y los objetivos planteados (0%) | La redacción de las conclusiones es desorganizada o confusa (8%) | Las conclusiones del trabajo son claras y hacen referencia al trabajo desarrollado y los objetivos planteados (15%) | 

## Introducción

### ROS2

ROS 2 (Robot Operating System 2) es un conjunto de bibliotecas y herramientas que permiten desarrollar software para robots de forma modular, distribuida y escalable.
En ROS 2, los programas se comunican entre sí mediante mensajes que viajan sobre una red (incluso entre computadoras distintas), usando un sistema de comunicación llamado DDS (Data Distribution Service).

<img src="imagenes/ROS2_logo.webp" alt = "ROS2" height="100" display= "block"/>

#### Nodos

Un nodo es la unidad básica de ejecución en ROS 2.
Cada nodo es un programa que realiza una tarea específica dentro del sistema del robot: leer un sensor, controlar un motor, procesar imágenes, etc.
En Python, los nodos se definen a partir de la clase base rclpy.node.Node.
~~~python
from rclpy.node import Node

class Nodo(Node):
    def __init__(self):
        super().__init__('nombre_nodo')
        self.get_logger().info("Nodo iniciado")
~~~

#### Tópicos

Los tópicos (topics) son canales de comunicación asíncrona y continua entre nodos.
Un nodo publica mensajes en un tópico, y otros nodos se suscriben para recibirlos.
Todos los mensajes que viajan por un tópico tienen un tipo de dato definido, por ejemplo:
- std_msgs/msg/String
- geometry_msgs/msg/Twist

Por ejemplo:

- Un nodo A (sensor) publica temperaturas en el tópico /temperatura.
- Otro nodo B (monitor) se suscribe a /temperatura y muestra los valores.

Para generar un publicador, se crea un publicador definiendo el tipo de mensaje y el tópico por el que se publica. Después se manda a llamar al publicador para mandar un mensaje.
~~~python
  topic_name = "/topic"
  self.publisher = self.create_publisher(String, topic_name, 10)

  msg = String()
  msg.data = "Message"
  self.publisher.publish(msg)
~~~
Para generar un subscriptor:
~~~python 
  topic_name = "/topic"
  self.subscription = self.create_subscription(
    String, topic_name, self.listener_callback, 10)
~~~

#### Servicios

Los servicios (services) son una forma de comunicación sincrónica (de solicitud–respuesta). Funcionan como una llamada a función entre nodos: uno envía una solicitud (request), y el otro responde con un resultado (response).

Los servicios se usan para operaciones puntuales, no continuas (por ejemplo, mover un robot a una posición, tomar una foto, guardar datos, etc.).

Por ejemplo:

- Un nodo A (cliente) pide: “calcula la suma de 2 y 3”.
- Otro nodo B (servidor) responde: “la suma es 5”.

Para crear un servidor, hay que crear el objeto y la función que se llama al recibir una solicitud

~~~python
  self.srv = self.create_service(AddTwoInts, server_name, self.add_two_ints_callback)
def add_two_ints_callback(self, request:AddTwoInts.Request, response:AddTwoInts.Response):
  response.sum = request.a + request.b
  self.get_logger().info("Recibido: a={}, b={}, respuesta={}".format(request.a, request.b, response.sum))
  return response
~~~

Para crear un cliente, hay que crear el objeto del cliente y la solicitud, enviarla, y definir la función que se llamará al recibir el objeto de la respuesta
~~~python
self.client = self.create_client(AddTwoInts, service_name)
# Esperamos al servicio
while not self.client.wait_for_service(timeout_sec=1.0):
  self.get_logger().info('Esperando al servicio {}...'.format(service_name))
request = AddTwoInts.Request()
request.a = self.a
request.b = self.b
# --- Manda solicitud
future = self.client.call_async(request)
# --- Función que se ejecuta al recibir respuesta
future.add_done_callback(self.callback_result)
~~~

#### Parámetros

Los parámetros son valores configurables que un nodo puede leer o modificar durante su ejecución.
Permiten ajustar el comportamiento del nodo sin cambiar el código.

Por ejemplo:

- Un nodo de control de motores puede tener un parámetro max_velocity de 0.1.
- Otro nodo puede cambiarlo a 0.5 sin interrumpir al primer nodo.

Por ejemplo, si en un nodo queremos que el tópico utilizado sea un parámetro
~~~python
self.declare_parameter("topic_param", "/topic")
topic_name = self.get_parameter("topic_param").value
~~~

Una forma gráfica de ver las comunicaciones en ROS2:

<img src="imagenes/Nodes-TopicandService.gif" height = "400" display = "block" alt = "Vosualización de las comunicaciones en ROS2"/>

### URDF (Unified Robot Description Format):

El URDF es un formato de archivo XML que se utiliza para describir la estructura de un robot, comunmente usado en ROS. Permite definir la composición física de un robot, incluyendo sus eslabones y juntas. Además, permite especificar características como el peso, la geometría, y la disposición espacial de los elementos del robot.

Un archivo URDF básico se construye a partir de **eslabones** y **juntas**.

#### Para definir un eslabón, se utiliza la estructura siguiente:
~~~xml
<!--Definición del eslabón-->
<link name = "nombre_del_eslabón">
  <!--
  Elementos visuales del eslabón 
  (modelos o formas geométricas)
  -->
  <visual>
      <!--
      Posición y rotación del elemento visual
      respecto al origen del eslabón
      -->
      <origin xyz = "x y z" rpy = "gamma beta alfa"/> 
        <!--Elementos visuales asociados a la junta-->
        <geometry> 
        <!--Primitiva visual (caja en este caso)-->
        <box size = "x y z"/> 
     </geometry>
        <!--Material usado para el elemento visual-->
        <material name = "nombre_del_material"> 
          <color rgba = "r g b a"/> 
        </material> 
  </visual> 
</link> 
~~~

Los elementos primitivos son formas geométricas básicas que se utilizan para representar las partes de un robot en URDF. Algunos de los más comunes son:

- **Caja (box)**: Un paralelepípedo definido por su ancho, alto y profundidad.
>~~~xml
>  <box size = "x y z"/> 
>~~~
- **Cilindro (cylinder)**: Definido por su radio y longitud.
>~~~xml
>  <cylinder radius = "r" length="l"/> 
>~~~
- **Esfera (sphere)**: Definida por su radio.
>~~~xml
>  <sphere radius = "r"/> 
>~~~
- **Malla (mesh)**: Permite importar geometrías más complejas en formatos como STL, Collada, obj, entre otros.
>~~~xml
>  <mesh filename = "ruta/al/archivo"/> 
>~~~

Se puede agregar cómo se desplegarán estos elementos a través de un **material**: 
- **Color**: Color plano con el que se despliega el elemento, definido por componentes rgb y transparencia (alfa).
>~~~xml
> <material name = "Nombre"> 
>   <color rgba = "r g b a"/> 
> </material> 
>~~~
- **Textura**: Imagen que se despliega sobre el modelo.
>~~~xml
> <material name = "Nombre"> 
>   <texture filename = "ruta/al/archivo"/> 
> </material> 
>~~~

#### Para una junta: 
~~~xml
<!--Definición de la junta-->
<joint name = "nombre_de_la_junta" type = "tipo_de_junta"> 
  <!--
  Eslabones padre e hijo
  - Padre: Eslabón respecto al cual se mide la rotació o traslación
  - Hijo: Eslabón que se desplaza o rota
  -->
  <parent link = "eslabon_padre_(fijo)"/> 
  <child link = "eslabon_hijo_(movible)"/> 
  <origin xyz = "x y z" rpy = "gamma beta alfa"/> 
  <!--Eje de rotación  (1: se mueve. 0: no se mueve)-->
  <axis xyz = "x y z"/> 
  <!--Límites 
  - effort: par máximo
  - velocity: velocidad absoluta máxima
  - lower-upper: posición mínima y máxima-->
  <limit effort="10.0" lower="-3.14" upper="3.14" velocity="3.14"/>
</joint>
~~~
En el formato URDF, existen varios tipos de juntas que pueden definir el movimiento relativo entre las partes de un robot:

- **Junta fija**: No permite ningún movimiento entre los elementos conectados.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "fixed"> 
>~~~
- **Junta rotacional**: Permite un movimiento de rotación alrededor de un eje.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "revolute"> 
>~~~
- **Junta prismática**: Permite un movimiento lineal a lo largo de un eje.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "prismatic"> 
>~~~
- **Junta continua**: Es una variación de la junta rotacional que permite una rotación sin límites.
>~~~xml
> <joint name = "nombre_de_la_junta" type = "continuous"> 
>~~~

### RVIZ


RViz (ROS Visualization) es una herramienta gráfica dentro del ecosistema de ROS que permite visualizar una amplia gama de información generada por robots. Esta herramienta es esencial para depurar, entender y visualizar los datos que se generan en tiempo real durante la simulación o la operación de un robot en un entorno físico.

<img src="imagenes/rviz.png" alt = "RViz" height="300" display= "block"/>


RViz soporta muchos tipos de visualizaciones, en este caso veremos:

- **Modelos 3D del robot (URDF)**: Puedes cargar y visualizar la descripción del robot usando su archivo URDF (Unified Robot Description Format). Esto te permite ver un modelo 3D del robot y observar cómo se mueve cada parte del robot (es decir, los eslabones y las juntas). También puedes ver cómo se actualiza en tiempo real cuando el robot cambia de posición.

- **Transformaciones (TF)**: ROS utiliza un sistema de coordenadas llamado TF (transformación de marcos) para definir la relación espacial entre diferentes partes del robot y el entorno. En RViz, puedes visualizar estos sistemas de referencia y las transformaciones entre ellos, lo que es útil para entender cómo se mueven y orientan los diferentes componentes del robot.
Esto incluye visualizar el origen de coordenadas del robot y sus partes móviles.

## Cuestionario previo

### Responder de forma breve las siguientes preguntas:

- ¿Qué es un ejecutor (executor) en ROS2?
> **Un ejecutor en ROS2 es un componente responsable de la gestión de la ejecución de callbacks asociados a nodos, tales como suscripciones, temporizadores, servidores de servicios y acciones. Un ejecutor utiliza uno o más hilos del sistema operativo para invocar estos callbacks cuando llegan mensajes o eventos. La clase en específico para el ejecutor en executor.hpp es rclcpp, in executors.py es rclpy y en executor.h es rclc [1].**
- ¿Qué es la ejecución por hilos en python?
> **La ejecución de hilos en python funciona mediante el módulo threading en python provee una forma de correr multiples hilos, concurrentemente en un sólo proceso, permitiendo así la creación y administración de hilos, lo que permite ejecutar distintas tareas en paralelo a traveś del reparto de espacio de memoria. El uso de hilos es particularmente útil con operaciones con archivos y haciendo peticiones de red, donde la mayoría del tiempo se va en esperar recursos externos [2].**
- ¿Qué ventajas tiene ejecutar un programa en un hilo secundario?
> **En base a lo investigado, ejecutar un programa en un hilo secundario, dada la ejecución concurrente de tareas, mientras un hilo realiza una tarea que consume bastante tiempo, el otro hilo puede ejecutar otra tarea sin bloquearse, lo que mejora la eficiencia y la capacidad de respuesta de otros programas donde sea necesaria la ejecución de interfaces de usuario, procesamiento de datos o bien comunicación con una red.**
- ¿Qué pasaría si se utiliza un comando que se queda en periodo de espera (como rclpy.spin()) dentro de un entorno de celdas como jupyter?
> **Sin la ejecución de múltiples hilos, todos los recursos computacionales para un proceso se estarían enfocando en la ejecución de la celda con ese comando, sin la posibilidad de poder ejecutar otra acción o servicio perteneciente a otra celda. En cambio, si se emplea en módulo de python para multi-hilos, al ejecutar la celda con este comando, los resursos computacionales del procesador se repartiran a fin de ejecutar la instrucción del ejecutor, mientras se ejecutan las instrucciones pertenecientes a otra celda, debido a que ya se le asigno otro espacio de memoria para efectuar la tarea al mismo tiempo.**

## Desarrollo

### 1. Tópicos en ROS2

En este primera parte, se usarán las funciones básicas de ROS2 dentro de jupyter

Para ello, en celdas separadas se realizarán las siguientes acciones:

- Inicializar ros2 e importar dependencias (nodos, mensajes, ejecución por hilos y ejecutor de ros2)
~~~python
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor
~~~
- Crear una clase que herede de nodo, que implemente un publicador y sus funciones
~~~python
  class Publisher(Node):
    def __init__(self):
      # Constructor de inicio
    def timer_callback(self):
      # Función de timer para publicar periódicamente cada segundo
~~~
- Crear una clase que herede de nodo, que implemente un subscriptor y sus funciones
~~~python
  class Subscriber(Node):
    def __init__(self):
      # Constructor
    def listener_callback(self, msg):
      # Función que se llama al recibir un mensaje
~~~
- Verificar si ROS2 está inicializado. Si no, inicializarlo. Utilizar la función rclpy.ok(), que nos devuelve el estado de ROS2
~~~python
  if not rclpy.ok():
    rclpy.init(args=None)
~~~
- Instanciar un ejecutor de ROS2 que se encargue de varios nodos de forma simultánea, y ejecutarlo en segundo plano para no bloquear la terminal
~~~ python
  executor = MultiThreadedExecutor()
  thread = threading.Thread(target=executor.spin, daemon=True)
  thread.start()
~~~
- Instanciar los nodos publicador y suscriptor y agregarlos al ejecutor
~~~python
  pub_node = SimplePublisher()
  sub_node = SimpleSubscriber()
  executor.add_node(sub_node)
  executor.add_node(pub_node)
~~~
- Destruir los nodos para que dejen de ejecutarse
~~~python
  pub_node.destroy_node()
  sub_node.destroy_node()
~~~
- Que detenga el proceso de ROS2 en caso de que siga activo (para evitar procesos en segundo plano)
~~~python
  if rclpy.ok():
    rclpy.shutdown()
~~~

Como probatorio: 
- Las celdas con código en esta sección del archivo
- Dejar los mensajes de salida que se imprimirán al agregar los nodos al ejecutor

In [13]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor

In [14]:
class Publisher(Node):
    def __init__(self):
        super().__init__('simple_publisher')
        topic_name = "/topic"
        self.publisher = self.create_publisher(String, topic_name, 10)
        self.timer = self.create_timer(1.0, self.timer_callback)
        self.get_logger().info(f"Publicador inciado en {topic_name}")
        self.count = 0

    def timer_callback(self):
        msg = String()
        msg.data = f"Mensaje número #{self.count}"
        self.publisher.publish(msg)
        self.get_logger().info(f"Publicación: {msg.data}")
        self.count += 1

class Subscriber(Node):
    def __init__(self):
        super().__init__('simple_subscriber')
        topic_name = "/topic"
        self.subscription = self.create_subscription(String, topic_name, self.listener_callback, 10)
        self.get_logger().info(f"Suscriptor escuchando tópico: {topic_name}")

    def listener_callback(self, msg):
        self.get_logger().info(f"Recibido: '{msg.data}' ")

In [21]:
if not rclpy.ok():
    rclpy.init(args=None)

In [22]:
executor = MultiThreadedExecutor()
thread = threading.Thread(target=executor.spin, daemon=True)
thread.start()

In [23]:
pub_node = Publisher()
sub_node = Subscriber()
executor.add_node(sub_node)
executor.add_node(pub_node)

[INFO] [1762554443.034560687] [simple_publisher]: Publicador inciado en /topic
[INFO] [1762554443.071456127] [simple_subscriber]: Suscriptor escuchando tópico: /topic


True

[INFO] [1762554444.031566866] [simple_publisher]: Publicación: Mensaje número #0
[INFO] [1762554444.039316951] [simple_subscriber]: Recibido: 'Mensaje número #0' 
[INFO] [1762554445.039704512] [simple_publisher]: Publicación: Mensaje número #1
[INFO] [1762554445.062566212] [simple_subscriber]: Recibido: 'Mensaje número #1' 
[INFO] [1762554446.047587020] [simple_publisher]: Publicación: Mensaje número #2
[INFO] [1762554446.073268386] [simple_subscriber]: Recibido: 'Mensaje número #2' 
[INFO] [1762554447.072191835] [simple_publisher]: Publicación: Mensaje número #3
[INFO] [1762554447.088676904] [simple_subscriber]: Recibido: 'Mensaje número #3' 
[INFO] [1762554448.079763752] [simple_subscriber]: Recibido: 'Mensaje número #4' 
[INFO] [1762554448.088925694] [simple_publisher]: Publicación: Mensaje número #4
[INFO] [1762554449.070951667] [simple_publisher]: Publicación: Mensaje número #5
[INFO] [1762554449.088803837] [simple_subscriber]: Recibido: 'Mensaje número #5' 
[INFO] [1762554450.054

In [24]:
pub_node.destroy_node()
sub_node.destroy_node()
if rclpy.ok():
    rclpy.shutdown()

*Fin de la primera sección "Topics in ROS2"*

### 2. Servicios en ROS2

En este segunda parte, se usarán los servicios de ROS2 dentro de jupyter

Para ello, en celdas separadas se realizarán las siguientes acciones:

- Inicializar ros2 e importar dependencias (nodos, mensajes, ejecución por hilos y ejecutor de ros2)
~~~python
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor
~~~
- Crear una clase que herede de nodo, que implemente un servidor y sus funciones
~~~python
  class Server(Node):
    def __init__(self):
      # Constructor
    def listener_callback(self, msg):
      # Función que se llama al recibir un mensaje
~~~
- Crear una clase que herede de nodo, que implemente un cliente y sus funciones
~~~python
  class Client(Node):
    def __init__(self):
      # Constructor de inicio
    def timer_callback(self):
      # Función de timer para enviar una solicitud periódicamente cada segundo
    def callback_result(self, future:Future):
      # Función que se llama al recibir una respuesta a la solicitud enviada
~~~
- Verificar si ROS2 está inicializado. Si no, inicializarlo. Utilizar la función rclpy.ok(), que nos devuelve el estado de ROS2
~~~python
  if not rclpy.ok():
    rclpy.init(args=None)
~~~
- Instanciar un ejecutor de ROS2 que se encargue de varios nodos de forma simultánea, y ejecutarlo en segundo plano para no bloquear la terminal
~~~ python
  executor = MultiThreadedExecutor()
  thread = threading.Thread(target=executor.spin, daemon=True)
  thread.start()
~~~
- Instanciar los nodos servidor y cliente, y agregarlos al ejecutor
~~~python
  server_node = Server()
  client_node = Client()
  executor.add_node(server_node)
  executor.add_node(client_node)
~~~
- Destruir los nodos para que dejen de ejecutarse
~~~python
  server_node.destroy_node()
  client_node.destroy_node()
~~~
- Que detenga el proceso de ROS2 en caso de que siga activo (para evitar procesos en segundo plano)
~~~python
  if rclpy.ok():
    rclpy.shutdown()
~~~

Como probatorio: 
- Las celdas con código en esta sección del archivo
- Dejar los mensajes de salida que se imprimirán al agregar los nodos al ejecutor

In [57]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import threading
from rclpy.executors import MultiThreadedExecutor
from example_interfaces.srv import AddTwoInts
from rclpy.task import Future

In [63]:
class Client(Node):
    def __init__(self):
        super().__init__('simple_service_client')
        service_name = '/sum_two_ints'
        self.a = 23
        self.b = 14
        self.client = self.create_client(AddTwoInts, service_name)
        while not self.client.wait_for_service(timeout_sec=1.0):
            self.get_logger().info(f"Esperando servicio de {service_name} ...")
        self.timer = self.create_timer(1.0, self.timer_callback)
        self.get_logger().info(f"Cliente iniciado a {service_name}")
        self.count = 0

    def timer_callback(self):
        self.count += 1
        request = AddTwoInts.Request()
        request.a = self.a
        request.b = self.b + self.count
        future = self.client.call_async(request)
        future.add_done_callback(self.callback_result)
        
    def callback_result(self, future:Future):
        try:
            response = future.result()
            response:AddTwoInts.Response
            self.get_logger().info(f"Resultado recibido: {self.a} + {self.a} + {response.sum}")
        except Exception as e:
            self.get_logger().error(f"Error al llamar al servicio: {e}")

class Server(Node):
    def __init__(self):
        super().__init__('simple_service_server')
        server_name = '/sum_two_ints'
        self.srv = self.create_service(AddTwoInts, server_name, self.add_two_ints_callback)
        self.get_logger().info(f"Servicio disponible en {server_name}")

    def add_two_ints_callback(self, request:AddTwoInts.Request, response:AddTwoInts.Response):
        response.sum = request.a + request.b
        self.get_logger().info(f"Recibido: num1={request.a}, num2={request.b}, respuesta={response.sum}")
        return response

In [64]:
if not rclpy.ok():
    rclpy.init(args=None)

In [65]:
executor = MultiThreadedExecutor()
thread = threading.Thread(target=executor.spin, daemon=True)
thread.start()

In [66]:
server_node = Server()
client_node = Client()

[INFO] [1762561163.553920066] [simple_service_server]: Servicio disponible en /sum_two_ints
[WARN] [1762561163.554666288] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.
[INFO] [1762561163.644882276] [simple_service_client]: Cliente iniciado a /sum_two_ints


In [67]:
executor.add_node(server_node)
executor.add_node(client_node)

True

[INFO] [1762561175.281933480] [simple_service_server]: Recibido: num1=23, num2=15, respuesta=38
[INFO] [1762561175.341227148] [simple_service_client]: Resultado recibido: 23 + 23 + 38
[INFO] [1762561175.653578520] [simple_service_server]: Recibido: num1=23, num2=16, respuesta=39
[INFO] [1762561175.717701655] [simple_service_client]: Resultado recibido: 23 + 23 + 39
[INFO] [1762561176.651123587] [simple_service_server]: Recibido: num1=23, num2=17, respuesta=40
[INFO] [1762561176.675564902] [simple_service_client]: Resultado recibido: 23 + 23 + 40
[INFO] [1762561177.628212913] [simple_service_server]: Recibido: num1=23, num2=18, respuesta=41
[INFO] [1762561177.653052877] [simple_service_client]: Resultado recibido: 23 + 23 + 41
[INFO] [1762561178.631008612] [simple_service_server]: Recibido: num1=23, num2=19, respuesta=42
[INFO] [1762561178.649389580] [simple_service_client]: Resultado recibido: 23 + 23 + 42
[INFO] [1762561179.633361030] [simple_service_server]: Recibido: num1=23, num2=2

In [68]:
server_node.destroy_node()
client_node.destroy_node()

if rclpy.ok():
    rclpy.shutdown()

*Fin de la segunda sección "Services in ROS2"*

### 2. URDF y RViz

En esta sección, se creará un archivo URDF de un robot tipo RRR con la siguiente disposición:
<div align="center">
<img src="imagenes/modelo_robot.png" alt = "Robot RRR" width="300" height="300"/>
</div>

** Se pueden utilizar formas geométricas sencillas
** Mantener las dimensiones menores a 0.5 \[m]. 

#### Despliegue de un modelo URDF con configuraciones predeterminadas en RViz
Ahora, se tomará el archivo URDF realizado y se desplegará en RViz utilizando algunas configuraciones predeterminadas. 
Para desplegar el archivo en RViz, se deben instalar algunas librerías:

Para descargar un paquete que contiene un launch para desplegar al robot
> $ sudo apt-get install ros-humble-urdf-tutorial

Instalar la librería que permite a ROS manipular los URDF (xacro)
> $ sudo apt install ros-humble-xacro

Hay que verificar las juntas del archivo. RViz requiere que las juntas tengan límites definidos, agregando el parámetro
``` xml 
<limit effort="XX" velocity="XX" lower="XX" upper="XX" />
```
Siendo "XX" los valores deseados para cada parámetro.

Con estos cambios, se puede correr el despliegue del modelo con un archivo *.launch del paquete urdf-tutorial 

> $ ros2 launch urdf_tutorial display.launch.py model:=/home/robousr/<ruta_del_modelo>/<nombre_del_modelo>.urdf



Como entregable, agregar una imagen del modelo desplegado en RViz y una celda con el código xml del modelo urdf


<div align="center">
<img src="imagenes/robot_rrp.png" alt = "Despliegue de modelo de robot rrp en RViz" width="800" height="800" display= "block"/>
</div>    

*Figura 1. Modelo desplegado de modelo de robot de tres grados de libertad, dos rotacionales y una prismática en RViz*


Contenido de archivo *robot_rrp.urdf*:

``` xml 
<robot name="robot_rrp">

	<!-- * * * Link Definitions * * * -->

 	<link name="base_link">
		<visual>
		    <origin xyz="0 0 0.075" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.3" length="0.15"/>
			</geometry>
			<material name="Black">
	       		<color rgba="0 0 0 1.0"/>
	     	</material>
		</visual>	
	</link>

 	<link name="column_link">
		<visual>
		    <origin xyz="0 0 0.2" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.1" length="0.4"/>
			</geometry>
			<material name="Cyan">
	       		<color rgba="0 0.9 0.9 1.0"/>
	     	</material>
		</visual>	
	</link>
	
	<link name="column_arm_circ1_link">
		<visual>
		    <origin xyz="0 0 0.05" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.1" length="0.1"/>
			</geometry>
			<material name="Cyan">
	       		<color rgba="0 0.9 0.9 1.0"/>
	     	</material>
		</visual>	
	</link>
	<link name="column_arm_link">
		<visual>
		    <origin xyz="0.225 0 0.05" rpy="0 0 0"/>
			<geometry>
				<box size="0.45 0.2 0.1"/>
			</geometry>
			<material name="Cyan">
	       		<color rgba="0 0.9 0.9 1.0"/>
	     	</material>
		</visual>	
	</link>
	<link name="column_arm_circ2_link">
		<visual>
		    <origin xyz="0 0 0.05" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.1" length="0.1"/>
			</geometry>
			<material name="Cyan">
	       		<color rgba="0 0.9 0.9 1.0"/>
	     	</material>
		</visual>	
	</link>

	<link name="second_arm_circ1_link">
		<visual>
		    <origin xyz="0 0 0.05" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.1" length="0.1"/>
			</geometry>
			<material name="Purple">
	       		<color rgba="0.5 0 1.0 1.0"/>
	     	</material>
		</visual>	
	</link>
	<link name="second_arm_link">
		<visual>
		    <origin xyz="0.225 0 0.05" rpy="0 0 0"/>
			<geometry>
				<box size="0.45 0.2 0.1"/>
			</geometry>
			<material name="Purple">
	       		<color rgba="0.5 0 1.0 1.0"/>
	     	</material>
		</visual>	
	</link>
	<link name="second_arm_circ2_link">
		<visual>
		    <origin xyz="0 0 0.05" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.1" length="0.1"/>
			</geometry>
			<material name="Purple">
	       		<color rgba="0.5 0 1.0 1.0"/>
	     	</material>
		</visual>	
	</link>

	<link name="column_final_effector_link">
		<visual>
		    <origin xyz="0 0 -0.1" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.075" length="0.2"/>
			</geometry>
			<material name="Purple">
	       		<color rgba="0.5 0 1.0 1.0"/>
	     	</material>
		</visual>	
	</link>
	<link name="final_effector_link">
		<visual>
		    <origin xyz="0 0 -0.15" rpy="0 0 0"/>
			<geometry>
				<cylinder radius="0.05" length="0.3"/>
			</geometry>
			<material name="White">
	       		<color rgba="1.0 1.0 1.0 1.0"/>
	     	</material>
		</visual>	
	</link>

	
	<!-- * * * Joint Definitions * * * -->
	
	<joint name="column_base_joint" type="revolute">
    	<parent link="base_link"/>
    	<child link="column_link"/>
    	<origin xyz="0 0 0.15" rpy="0 0 0"/>
    	<axis xyz="0 0 1"/>
    	<limit lower="-3.1416" upper="3.1416" effort="10" velocity="3"/>
  	</joint>
 
	<joint name="column_arm_circ1_joint" type="fixed">
    	<parent link="column_link"/>
    	<child link="column_arm_circ1_link"/>
    	<origin xyz="0 0 0.4" rpy="0 0 0"/>
	</joint>
	<joint name="column_arm_joint" type="fixed">
    	<parent link="column_arm_circ1_link"/>
    	<child link="column_arm_link"/>
    	<origin xyz="0.0 0 0" rpy="0 0 0"/>
	</joint>
	<joint name="column_arm_circ2_joint" type="fixed">
    	<parent link="column_arm_link"/>
    	<child link="column_arm_circ2_link"/>
    	<origin xyz="0.45 0 0" rpy="0 0 0"/>
	</joint>

	<joint name="second_and_column_arm_circ1_joint" type="revolute">
    	<parent link="column_arm_circ2_link"/>
    	<child link="second_arm_circ1_link"/>
    	<origin xyz="0 0 0.1" rpy="0 0 0"/>
    	<axis xyz="0 0 1"/>
    	<limit lower="-2.67" upper="2.67" effort="10" velocity="3"/>
	</joint>
	<joint name="second_arm_joint" type="fixed">
    	<parent link="second_arm_circ1_link"/>
    	<child link="second_arm_link"/>
    	<origin xyz="0.0 0 0" rpy="0 0 0"/>
	</joint>
	<joint name="second_arm_circ2_joint" type="fixed">
    	<parent link="second_arm_link"/>
    	<child link="second_arm_circ2_link"/>
    	<origin xyz="0.45 0 0" rpy="0 0 0"/>
	</joint>

	<joint name="secondArm_colFinalEffector_joint" type="fixed">
    	<parent link="second_arm_circ2_link"/>
    	<child link="column_final_effector_link"/>
    	<origin xyz="0 0 0" rpy="0 0 0"/>
	</joint>
	<joint name="column_final_effector_joint" type="prismatic">
    	<parent link="column_final_effector_link"/>
    	<child link="final_effector_link"/>
    	<origin xyz="0.0 0 0" rpy="0 0 0"/>
    	<axis xyz="0 0 1"/>
    	<limit lower="0" upper="-0.15" effort="10" velocity="3"/>
	</joint>

</robot>

```

## Análisis de resultados

¿Cuáles son las ventajas o desventajas de usar el protocolo de comunicación publicador/suscriptor ó cliente/servicio?
> **La ventaja es que dicha comunicación puede hacerse entre diferentes sistemas aunque estén separados físicamente para la difusión de datos por parte de un publicador, entre los nodos que componen la red de suscriptores. Sin embargo, una desventaja tal como los protocolos de internet, es la posible pérdida de mensajes, el tiempo de espera de las peticiones o bien que la ejecución de un servicio consuma bastantes recursos computacionales**

¿Cuál es la convención de ángulos que utilizan los archivos URDF para los ángulos de las juntas respecto al eslabón padre? (Intrínsecos/extrínsecos y el orden)
> **Los archivos URDF utilizan rotaciones intrínsecas, es decir, rotaciones respecto a los ejes del sistema local. El orden es el siguiente:**
> **rpy = roll-pitch-yaw = rotación intrínseca en el orden X → Y →** 

¿Qué utilidad tiene describir un robot en un archivo URDF?
> **Describir un robot en un archivo URDF sirve para que un gran número de simuladores desplieguen un modelo con una descripción común y funcional de sus grados de libertad, debido a que define las características de cada eslabón (geometría, posición, rotación, color, etc) y juntas (tipo de junta, posición, rotación, punto de origen, eje de movimiento, limites de parámetros, etc), lo cual permite generar una simulación del control de movimiento del robot mediante la cinemática directa.**




## Conclusiones

En base al procedimiento realizado y a los resultados obtenidos, se determino que el uso de nodos y servicios en ROS2 es esencial para establecer comunicación entre dos sistemas a fin de mandar o recibir información de manera eficiente y automatizandolo mediante scripts de python, dando la posibilidad de integrarse a otros sistemas informáticos y crear un proceso autónomo. Por otra parte, el uso y creación de los archivos .urdf son esenciales para la simulación y modelado de los robots en ROS2, ya que igual se puede integrar e este entorno de automatización mediante python, debido a que el modelo en el archivo urdf puede ser parametrizado en una simulación, lo que da a lugar a la definición de trayectorias, comportamientos o evaluaciones específicas para el comportamiento de un robot. Por lo que se pueden integrar todas estas herramientas para crear un sistema completo de control robótico.



## Bibliografía 

- [1] Open Robotics. (2025). *ROS 2 Documentation: Foxy. Executors*. Recuperado en noviembre 07, 2025 de docs.ros.org/en/foxy/Concepts/About-Executors.html
- [2] Python Software Foundation. (2001). *Threading - Thread - based parallelism*. Recuperado en noviembre de 07, 2025 de docs.python.org/es/3/library/threading.html